In [1]:
#Подгрузка необходимых библиотек

import pandas as pd
import numpy as np
from google.oauth2.service_account import Credentials

In [ ]:
#Подгрузка raw-data через excel уменьшает вероятность шума данных, они максимально сходятся с ежемесячными ручными сверками партнеров

In [2]:
#Подгрузка raw-data Appsflyer по FTD с файла Excel по iOS-устройствам по привлеченному трафику, внести адрес загрузки CSV

df_ios = pd.read_csv (r"C:\.csv")
prod_cols = {'Media Source': 'count'}

#Перевод в datetime времени ивента и времени аттрибуции массива  данных

event_time = pd.to_datetime(df_ios['Event Time']).dt.date
install_time = pd.to_datetime(df_ios['Install Time']).dt.date
date30 = np.int64(30)

#Выполнение условия по разнице времени ивента и времени инсталла — не должна превышать 30 дней включительно

df_ios['Is_true?'] = (event_time - install_time).dt.days - date30
data_final_ios = df_ios.drop(df_ios[df_ios['Is_true?'] >= 0].index)

#Составление pivot-table

pivot_ios_ftd = pd.pivot_table(data_final_ios, index = ['Partner', 'Media Source'], columns=event_time, aggfunc=prod_cols)

In [3]:
#Подгрузка raw-data Appsflyer по FTD с файла Excel по Android-устройствам по привлеченному трафику, внести адрес загрузки CSV

df_android = pd.read_csv (r"C:\")
prod_cols = {'Media Source': 'count'}

#Перевод в datetime времени ивента и времени аттрибуции массива  данных

event_time = pd.to_datetime(df_android['Event Time']).dt.date
install_time = pd.to_datetime(df_android['Install Time']).dt.date
date30 = np.int64(30)

#Выполнение условия по разнице времени ивента и времени инсталла — не должна превышать 30 дней включительно

df_android['Is_true?'] = (event_time - install_time).dt.days - date30
data_final_android = df_android.drop(df_android[df_android['Is_true?'] > 0].index)

#Составление pivot-table

pivot_and_ftd = pd.pivot_table(data_final_android, index = ['Partner', 'Media Source'], columns=event_time, aggfunc=prod_cols)

In [4]:
#Подгрузка кредов для работы с DataLocker, внести свое название 

creds = Credentials.from_service_account_file('cred-name.json')

In [5]:
#Выполнение SQL-запроса по необходимому Workspace, install-time меняется 1 раз в месяц

sql = """
SELECT *
FROM `SQL-workspace-installs`
WHERE install_time >= '2023-07-01' AND af_prt IS NOT NULL 
"""

In [6]:
#В project_id подставить необходимое 

df_installs = pd.read_gbq(sql,
                project_id='datalake',
                credentials=creds)

In [7]:
#Перевод в datetime времени ивента и времени аттрибуции массива данных установок

prod_cols = {'af_prt': 'count'}
event_time = pd.to_datetime(df_installs['event_time']).dt.date
install_time = pd.to_datetime(df_installs['install_time']).dt.date
date30 = np.int64(30)

In [8]:
ins_platform_ios = df_installs.loc[(df_installs['platform'] == 'ios')]
ins_platform_android = df_installs.loc[(df_installs['platform'] == 'android')]

In [9]:
#Составление Power-pivot по установкам

pivot_ios_ins = pd.pivot_table(ins_platform_ios, index = ['af_prt', 'media_source'], columns=event_time, aggfunc=prod_cols)
pivot_and_ins = pd.pivot_table(ins_platform_android, index = ['af_prt', 'media_source'], columns=event_time, aggfunc=prod_cols)

In [10]:
#Выгрузка готовой эксели с разбивкой по iOS / Android

with pd.ExcelWriter('xlsx\pivot_daily_stat_jul.xlsx') as writer:  
    pivot_ios_ins.to_excel(writer, sheet_name='iOS installs')
    pivot_ios_ftd.to_excel(writer, sheet_name='iOS FTD')
    pivot_and_ins.to_excel(writer, sheet_name='Android installs')
    pivot_and_ftd.to_excel(writer, sheet_name='Android FTD')